In [1]:
import time
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install ultralytics

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.3/838.3 MB 5.4 MB/s eta 0:00:0000:0100:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 5.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 5.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 5.6 MB/s eta 0:00:0000:0100:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 5.6 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 5.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204

In [3]:
import os
import cv2
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import random
import yaml
from shutil import copyfile
from ultralytics import YOLO
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import average_precision_score
import numpy as np

In [4]:
base_path = "../resources/dataset"
annotations_path = os.path.join(base_path, "annotations")
images_path = os.path.join(base_path, "images")
train_output_path = "../resources/yolo_train"
test_output_path = "../resources/yolo_test"
model_output_path = "../resources/yolo_model"
yaml_config_path = "yolo_config.yaml"

In [13]:

def split_dataset(images_path, annotations_path, train_output_path, test_output_path, test_ratio=0.2, seed=42):
    """
    Splits a dataset of images and annotations into training and testing sets.

    Parameters:
        images_path (str): Path to the folder containing image files.
        annotations_path (str): Path to the folder containing annotation files.
        train_output_path (str): Path to save the training set.
        test_output_path (str): Path to save the testing set.
        test_ratio (float): Ratio of the dataset to be used for testing. Default is 0.2.
        seed (int): Random seed for reproducibility. Default is 42.

    Returns:
        None
    """
    # Collect valid image files that have corresponding annotation files
    image_files = [
        f for f in os.listdir(images_path)
        if f.endswith(('.png', '.jpg', '.jpeg')) and os.path.exists(
            os.path.join(annotations_path, os.path.splitext(f)[0] + '.xml')
        )
    ]

    # Shuffle files with a fixed random seed
    random.seed(seed)
    random.shuffle(image_files)

    # Calculate the split
    test_count = int(len(image_files) * test_ratio)
    test_files = image_files[:test_count]
    train_files = image_files[test_count:]

    # Ensure output directories exist
    for folder in [train_output_path, test_output_path]:
        os.makedirs(os.path.join(folder, "images"), exist_ok=True)
        os.makedirs(os.path.join(folder, "labels"), exist_ok=True)

    # Helper function to copy files
    def copy_files(file_list, output_path):
        for image_file in file_list:
            annotation_file = os.path.splitext(image_file)[0] + '.xml'
            image_src = os.path.join(images_path, image_file)
            annotation_src = os.path.join(annotations_path, annotation_file)

            if os.path.exists(annotation_src):
                copyfile(image_src, os.path.join(output_path, "images", image_file))
                copyfile(annotation_src, os.path.join(output_path, "labels", annotation_file))
            else:
                print(f"Warning: Missing annotation for {image_file}. Skipping.")

    # Copy files to train and test directories
    copy_files(train_files, train_output_path)
    copy_files(test_files, test_output_path)

    # Log the results
    print(f"Dataset split completed.")
    print(f"Train set: {len(train_files)} images")
    print(f"Test set: {len(test_files)} images")

In [6]:
def prepare_yolo_dataset(annotations_path, output_path):
    labels_path = os.path.join(output_path, "labels")
    images_path = os.path.join(output_path, "images")

    for annotation_file in os.listdir(labels_path):
        if annotation_file.endswith(".xml"):
            annotation_path = os.path.join(labels_path, annotation_file)

            tree = ET.parse(annotation_path)
            root = tree.getroot()

            # Extract image size
            size = root.find("size")
            image_width = int(size.find("width").text)
            image_height = int(size.find("height").text)

            # Create YOLO format labels
            label_file = os.path.join(labels_path, annotation_file.replace(".xml", ".txt"))
            with open(label_file, 'w') as label_out:
                for obj in root.findall("object"):
                    class_name = obj.find("name").text
                    class_id = 0 if class_name == "without_mask" else 1  # Map class names to IDs

                    bndbox = obj.find("bndbox")
                    xmin = int(bndbox.find("xmin").text)
                    ymin = int(bndbox.find("ymin").text)
                    xmax = int(bndbox.find("xmax").text)
                    ymax = int(bndbox.find("ymax").text)

                    x_center = ((xmin + xmax) / 2) / image_width
                    y_center = ((ymin + ymax) / 2) / image_height
                    width = (xmax - xmin) / image_width
                    height = (ymax - ymin) / image_height

                    label_out.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

In [7]:
def generate_yaml_config(train_path, test_path):
    config = {
        'train': os.path.abspath(os.path.join(train_path, 'images')),
        'val': os.path.abspath(os.path.join(test_path, 'images')),
        'nc': 2,  # Number of classes
        'names': ['without_mask', 'with_mask']  # Class names
    }
    with open(yaml_config_path, 'w') as yaml_file:
        yaml.dump(config, yaml_file)
    return yaml_config_path

In [8]:
def train_yolo_model(train_path, test_path, epochs=80, batch_size=16):
    model = YOLO("yolov8n.pt")  # Load a pre-trained YOLO model (YOLOv8 Nano as an example)

    # Generate YAML config file
    data_config_path = generate_yaml_config(train_path, test_path)

    # Ensure the YAML file is created using exponential backoff
    delay = 1  # Initial delay in seconds
    max_retries = 10
    for attempt in range(max_retries):
        if os.path.exists(data_config_path):
            break
        print(f"YAML file not found. Retrying in {delay} seconds...")
        time.sleep(delay)
        delay *= 2  # Double the delay for exponential backoff
    else:
        raise FileNotFoundError(f"YAML configuration file '{data_config_path}' was not created after {max_retries} attempts.")

    # Define training configuration
    model.train(
        data=data_config_path,
        epochs=epochs,
        batch=batch_size,  # Correct parameter name
        project=model_output_path,
        name="yolo_training",
        exist_ok=True
    )


In [9]:
def calculate_iou(box1, box2):
    """
    Calculate the Intersection over Union (IoU) of two boxes.
    :param box1: [x1, y1, x2, y2]
    :param box2: [x1, y1, x2, y2]
    :return: IoU value
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Calculate intersection area
    intersection = max(0, x2 - x1) * max(0, y2 - y1)

    # Calculate areas of the boxes
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Calculate union area
    union = area_box1 + area_box2 - intersection

    # IoU
    return intersection / union if union > 0 else 0

In [17]:
def evaluate_yolo_model_metrics(model_path, test_images_path, ground_truth_boxes_dict):
    model = YOLO(model_path)  # Load the trained YOLO model

    y_true = []  # True labels
    y_pred = []  # Predicted labels
    iou_scores = []  # List to store IoU scores
    conf_scores = []  # List to store confidence scores

    for image_file in os.listdir(test_images_path):
        if image_file.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(test_images_path, image_file)

            # Skip images with no ground truth
            if image_file not in ground_truth_boxes_dict:
                print(f"Warning: No ground truth for {image_file}. Skipping...")
                continue

            results = model.predict(source=image_path, save=False)

            # Get predicted boxes
            predicted_boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes.xyxy is not None else []
            predicted_classes = results[0].boxes.cls.cpu().numpy() if results[0].boxes.cls is not None else []
            predicted_confidences = results[0].boxes.conf.cpu().numpy() if results[0].boxes.conf is not None else []
            ground_truth_boxes = ground_truth_boxes_dict.get(image_file, [])  # Ground truth boxes

            # Check for mismatch
            if len(ground_truth_boxes) != len(predicted_boxes):
                print(f"Skipping {image_file}: Ground Truths={len(ground_truth_boxes)}, Predictions={len(predicted_boxes)}")
                continue

            # Add ground truth classes
            y_true.extend([gt_box[4] for gt_box in ground_truth_boxes])

            # Add predictions to y_pred
            for pred_class, pred_conf in zip(predicted_classes, predicted_confidences):
                y_pred.append(pred_class)
                conf_scores.append(pred_conf)

            # Debugging output
            print(f"Processed {image_file}: Ground Truths={len(ground_truth_boxes)}, Predictions={len(predicted_boxes)}")

    # Calculate metrics only if there are valid samples
    if not y_true or not y_pred:
        raise ValueError("No valid samples for evaluation. Check your data and model predictions.")

    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=1)
    mean_iou = np.mean(iou_scores) if iou_scores else 0
    mean_conf = np.mean(conf_scores) if conf_scores else 0

    # Print metrics
    print("\nEvaluation Metrics:")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Mean IoU: {mean_iou:.2f}")
    print(f"Mean Confidence: {mean_conf:.2f}")


In [14]:
def preprocess_data():
    print("Splitting dataset into training and testing sets...")
    split_dataset(images_path, annotations_path, train_output_path, test_output_path)

    print("Preparing YOLO datasets...")
    prepare_yolo_dataset(annotations_path, train_output_path)
    prepare_yolo_dataset(annotations_path, test_output_path)
    
    

# Run the main function
preprocess_data()

Splitting dataset into training and testing sets...
Dataset split completed.
Train set: 683 images
Test set: 170 images
Preparing YOLO datasets...


In [15]:
def train_model_and_eval():
    print("Training YOLO model...")
    train_yolo_model(train_output_path, test_output_path, epochs=80, batch_size=16)

    print("Evaluating YOLO model metrics...")
    evaluate_yolo_model_metrics(os.path.join(model_output_path, "yolo_training/weights/best.pt"), os.path.join(test_output_path, "images"))


    print("YOLO training and evaluation completed.")
train_model_and_eval()

Training YOLO model...


100%|██████████| 6.25M/6.25M [00:01<00:00, 4.96MB/s]


Ultralytics 8.3.58 🚀 Python-3.10.0 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5938MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=yolo_config.yaml, epochs=80, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=../resources/yolo_model, name=yolo_training, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show

100%|██████████| 5.35M/5.35M [00:01<00:00, 5.21MB/s]


AMP: checks passed ✅


...

                   all        170        933      0.894      0.844      0.912      0.598



80 epochs completed in 0.153 hours.
Optimizer stripped from ../resources/yolo_model/yolo_training/weights/last.pt, 6.2MB
Optimizer stripped from ../resources/yolo_model/yolo_training/weights/best.pt, 6.2MB

Validating ../resources/yolo_model/yolo_training/weights/best.pt...
Ultralytics 8.3.58 🚀 Python-3.10.0 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5938MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  4.77it/s]


                   all        170        933      0.903      0.825      0.911        0.6
          without_mask         56        192      0.859      0.734      0.859      0.531
             with_mask        164        741      0.947      0.915      0.963       0.67
Speed: 0.3ms preprocess, 1.6ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to ../resources/yolo_model/yolo_training
Evaluating YOLO model metrics...


TypeError: evaluate_yolo_model_metrics() missing 1 required positional argument: 'ground_truth_boxes_dict'

In [18]:
def parse_ground_truth_boxes(annotations_path):
    """
    Parse ground truth bounding boxes from XML files into a dictionary.
    :param annotations_path: Path to the annotations directory.
    :return: Dictionary mapping image filenames to bounding boxes.
    """
    ground_truth_boxes = {}
    for annotation_file in os.listdir(annotations_path):
        if annotation_file.endswith('.xml'):
            annotation_path = os.path.join(annotations_path, annotation_file)
            tree = ET.parse(annotation_path)
            root = tree.getroot()

            boxes = []
            for obj in root.findall("object"):
                class_name = obj.find("name").text
                class_id = 0 if class_name == "without_mask" else 1  # Map class names to IDs

                bndbox = obj.find("bndbox")
                xmin = int(bndbox.find("xmin").text)
                ymin = int(bndbox.find("ymin").text)
                xmax = int(bndbox.find("xmax").text)
                ymax = int(bndbox.find("ymax").text)

                boxes.append([xmin, ymin, xmax, ymax, class_id])

            image_filename = root.find("filename").text
            ground_truth_boxes[image_filename] = boxes

    return ground_truth_boxes
ground_truth_boxes_dict = parse_ground_truth_boxes(annotations_path)
evaluate_yolo_model_metrics(
    os.path.join(model_output_path, "yolo_training/weights/best.pt"),
    os.path.join(test_output_path, "images"),
    ground_truth_boxes_dict
)


image 1/1 /home/djentci/Desktop/comp_vision_project/src/../resources/yolo_test/images/maksssksksss196.png: 640x512 1 with_mask, 9.4ms
Speed: 2.1ms preprocess, 9.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 512)
Processed maksssksksss196.png: Ground Truths=1, Predictions=1

image 1/1 /home/djentci/Desktop/comp_vision_project/src/../resources/yolo_test/images/maksssksksss765.png: 640x512 1 with_mask, 6.4ms
Speed: 1.5ms preprocess, 6.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 512)
Processed maksssksksss765.png: Ground Truths=1, Predictions=1

image 1/1 /home/djentci/Desktop/comp_vision_project/src/../resources/yolo_test/images/maksssksksss776.png: 448x640 2 with_masks, 10.1ms
Speed: 2.0ms preprocess, 10.1ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)
Processed maksssksksss776.png: Ground Truths=2, Predictions=2

image 1/1 /home/djentci/Desktop/comp_vision_project/src/../resources/yolo_test/images/maksssksksss430.png: 480x640 1 